## Implementing search for our database

In [2]:
import minsearch

In [3]:
import json

In [5]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc['course'] = course_dict["course"]
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [11]:
q = "the course has already started, can I still join?"

In [12]:
index.fit(documents)

In [15]:
boost = {"question": 3.0, "section": 0.5}
filter_course = {"course": "data-engineering-zoomcamp"}
results = index.search(query=q, filter_dict=filter_course, boost_dict=boost, num_results=5)

In [16]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

## Prompt Engineering for LLM (uses Gemini API)

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()  # This line brings all environment variables from .env into os.environ

True

In [18]:
import google.generativeai as genai

/home/msvelan/.local/share/virtualenvs/LLM-dr_kSaRK/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [21]:
model = genai.GenerativeModel('gemini-1.5-flash')


In [24]:
# %%time
# response = model.generate_content("What is the meaning of life?")

In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [40]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [41]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [42]:
prompt = prompt_template.format(question=q, context=context).strip()

In [43]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still join?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only)

In [44]:
%%time
response = model.generate_content(prompt)

I0000 00:00:1721931626.885821  437675 subchannel.cc:806] subchannel 0x59922bbb0d70 {address=ipv6:%5B2404:6800:4002:819::200a%5D:443, args={grpc.client_channel_factory=0x59922bb55f50, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x59922bb987a0, grpc.internal.client_channel_call_destination=0x78ce47e4b3d0, grpc.internal.event_engine=0x59922bd394d0, grpc.internal.security_connector=0x59922ab2d1a0, grpc.internal.subchannel_pool=0x59922bb99290, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.1, grpc.resource_quota=0x59922bb99180, grpc.server_uri=dns:///generativelanguage.googleapis.com:443}}: connect failed (UNKNOWN:connect: Network is unreachable (101) {created_time:"2024-07-25T23:50:26.885714473+05:30"}), backing off for 999 ms


CPU times: user 6.11 ms, sys: 9.96 ms, total: 16.1 ms
Wall time: 2.74 s


In [46]:
print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. 
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

